In [1]:
import os
import glob
import numpy as np
import scipy.io
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import tensorflow_addons as tfa
import pandas as pd
import matplotlib.pyplot as plt
adam = tf.keras.optimizers.Adam(learning_rate=0.0005)
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Input, Conv1D, MaxPooling1D, Flatten, Dense
import numpy as np
import scipy.io as sio
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Reshape

from keras.callbacks import ModelCheckpoint ,EarlyStopping
temperature = 0.03
learning_rate=0.001

def dataloader(path, featType):
    """
    Load data from a MATLAB file.

    Args:
        path (str): Path to the MATLAB file.
        featType (int): Type of features to load.

    Returns:
        Tuple: Tuple containing input features, labels, weights, and additional information.
    """
    data = scipy.io.loadmat(path)
    print(data.keys())

    AF = data['AF']
    x1 = AF[:-2]
    y = AF[-2]
    w = AF[-1]

    if featType == 1:
        x = x1
    else:
        x2 = data['CF']
        x = np.concatenate((x1, x2), axis=0)
    return x.T, y.T, w.T, data['CF_info']

def calculate_accuracy(arr1, arr2):
    """
    Calculate the accuracy between two arrays.

    Args:
        arr1 (array): First array.
        arr2 (array): Second array.

    Returns:
        float: Accuracy between the two arrays.
    """
    count = sum(1 for itr1, itr2 in zip(arr1, arr2) if itr1 == itr2)
    return count / len(arr1)

def normalization(feats):

    """
    Normalize the input features using standard scaling.

    Args:
        feats (array): Input features.

    Returns:
        array: Normalized features.
    """
    df = pd.DataFrame(feats)
    scaler = StandardScaler()
    x_new = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    return x_new

def make_partitions(arr_words, arr_labels):

    """
    Create partitions based on word boundaries and labels.

    Args:
        arr_words (array): Array of words.
        arr_labels (array): Array of labels.

    Returns:
        array: Partitions based on word boundaries and labels.
    """
    v = []
    temp = []

    for i in range(len(arr_words) - 1):
        word = arr_words[i]
        next_word = arr_words[i + 1]
        temp.append(arr_labels[i])

        if word != next_word or i == len(arr_words) - 2:
            if i == len(arr_words) - 2:
                temp.append(arr_labels[i + 1])

            numpy_temp = np.array(temp)
            temp_max = np.amax(numpy_temp)
            numpy_temp = np.divide(numpy_temp, temp_max)
            v = np.concatenate((v, numpy_temp), axis=None)
            temp.clear()

    v1 = [1 if i == 1 else 0 for i in v]
    return v1
    
fatyp = 'TypicalFA_comb1'
drivepath = 'finalData/'+ fatyp +'/';
#featFiles = 'GER_train_fisher-2000_FA_GT_ESTphnTrans_estStress'               #glob.glob(drivepath + '*train*')
filee = drivepath+'GER_train_fisher-2000_FA_GT_ESTphnTrans_estStress.mat'
featType = 1; #Acoustic or Acoustic+context
if featType == 1:
  original_dim = 19
else:
  original_dim = 38


# print('Classification with::::::',os.path.basename(filee))

train_path = filee; test_path = filee.replace('train','test')
# print('test file:::::::',os.path.basename(test_path))
xtrain, ytrain, wtrain, info_train = dataloader(train_path, featType); 
xtrain1, ytrain1, wtrain1, info_train1 = dataloader(train_path, featType=2);
xtest, ytest, wtest ,info_test = dataloader(test_path, featType)
xtest1, ytest1, wtest1, info_test1 = dataloader(test_path, featType=2);

xtest_a = normalization(xtest)
xtest_ac = normalization(xtest1)
xtrain = normalization(xtrain)
xtrain1 = normalization(xtrain1)

woPP=[]; wPP=[]
input_shape1 = (19,1)
input_shape2 = (38,1)
temperature = 0.03
learning_rate=0.

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
# Splitting xtrain and ytrain into training and validation sets
xtra_a, xval_a, ytra_a, yval_a = train_test_split(xtrain, ytrain, test_size=0.2, random_state=42)

# Splitting xtrain1 and ytrain1 into training and validation sets
xtra_ac, xval_ac, ytra_ac, yval_ac = train_test_split(xtrain1, ytrain1, test_size=0.2, random_state=42)

learning_rate = 0.0005
batch_size = 16
hidden_units = 64
projection_units = 128
num_epochs = 200
dropout_rate = 0.3
num_classes = 2
input_shape1 = (19,)
input_shape = (38,)

2023-06-08 06:00:12.316594: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/project/.local/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
2023-06-08 06:00:16.789605: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but 

dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])
dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])
dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])
dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])


In [2]:
def create_classifier(hidden_units):

    # for layer in encoder.layers:
    #     layer.trainable = trainable

    inputs = keras.Input(shape=input_shape)
    # features = encoder(inputs)
    features=layers.Dense(hidden_units, activation="relu")(inputs)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units//2, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units//2, activation="relu")(features)
    features = layers.Dense(hidden_units//2, activation="relu")(features)
    features = layers.Dense(hidden_units//2, activation="relu")(features)
    features = layers.Dense(hidden_units//2, activation="relu")(features)
    outputs = layers.Dense(1, activation="sigmoid")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="classifier")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=[keras.metrics.BinaryAccuracy()],
    )
    return model

units=64

classifier = create_classifier(units)

# history = classifier.fit(x=xtra_ac, y=ytra_ac, validation_data =(xval_ac,yval_a), batch_size=batch_size, epochs=num_epochs, callbacks=[EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)])


In [3]:
classifier.summary()
classifier.fit(x=xtra_ac, y=ytra_ac, validation_data =(xval_ac,yval_a), batch_size=batch_size, epochs=num_epochs, callbacks=[EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)])

Model: "classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 38)]              0         
                                                                 
 dense (Dense)               (None, 64)                2496      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                1056      
                                                                 
 dense_3 (Dense)             (None, 32)                1

/usr/local/lib/python3.8/dist-packages/keras/backend.py:5703: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2023-06-08 06:01:23.037078: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x21750420 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-06-08 06:01:23.037119: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti, Compute Capability 7.5
2023-06-08 06:01:23.078425: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-06-08 06:01:23.465049: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-06-08 06:01:23.808974: I ./tensorflow/compiler/jit/device_compi

290/295 [============================>.] - ETA: 0s - loss: 0.6032 - binary_accuracy: 0.6685

/usr/local/lib/python3.8/dist-packages/keras/backend.py:5703: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


295/295 [==============================] - 7s 7ms/step - loss: 0.6032 - binary_accuracy: 0.6689 - val_loss: 0.5147 - val_binary_accuracy: 0.7447
Epoch 2/200
295/295 [==============================] - 2s 6ms/step - loss: 0.5209 - binary_accuracy: 0.7572 - val_loss: 0.4793 - val_binary_accuracy: 0.7795
Epoch 3/200
295/295 [==============================] - 2s 6ms/step - loss: 0.4922 - binary_accuracy: 0.7691 - val_loss: 0.4565 - val_binary_accuracy: 0.8007
Epoch 4/200
295/295 [==============================] - 2s 6ms/step - loss: 0.4714 - binary_accuracy: 0.7782 - val_loss: 0.4441 - val_binary_accuracy: 0.8066
Epoch 5/200
295/295 [==============================] - 2s 6ms/step - loss: 0.4507 - binary_accuracy: 0.7975 - val_loss: 0.4247 - val_binary_accuracy: 0.8185
Epoch 6/200
295/295 [==============================] - 2s 6ms/step - loss: 0.4310 - binary_accuracy: 0.8092 - val_loss: 0.4108 - val_binary_accuracy: 0.8253
Epoch 7/200
295/295 [==============================] - 2s 6ms/step - l

In [97]:
from sklearn.metrics import accuracy_score

y_pred = classifier.predict(xtest_ac)
y_pred = np.round(y_pred).flatten()
accuracy = accuracy_score(ytest, y_pred)

from sklearn.metrics import f1_score

f1 = f1_score(ytest, y_pred)

print("Accuracy:", accuracy)
print("F1 Score:", f1)

150/150 [==============================] - 0s 1ms/step
Accuracy: 0.8993526832324076
F1 Score: 0.8856193640246796


### For Italian

In [98]:
fatyp = 'TypicalFA_comb1'
drivepath = 'finalData/'+ fatyp +'/';
#featFiles = 'GER_train_fisher-2000_FA_GT_ESTphnTrans_estStress'               #glob.glob(drivepath + '*train*')
filee = drivepath+'ITA_train_fisher-2000_FA_GT_ESTphnTrans_estStress.mat'
featType = 1; #Acoustic or Acoustic+context
if featType == 1:
  original_dim = 19
else:
  original_dim = 38

# print('Classification with::::::',os.path.basename(filee))

train_path = filee; test_path = filee.replace('train','test')
# print('test file:::::::',os.path.basename(test_path))
xtrain, ytrain, wtrain, info_train = dataloader(train_path, featType); 
xtrain1, ytrain1, wtrain1, info_train1 = dataloader(train_path, featType=2);
xtest, ytest, wtest ,info_test = dataloader(test_path, featType)
xtest1, ytest1, wtest1, info_test1 = dataloader(test_path, featType=2);


xtest_a = normalization(xtest)
xtest_ac = normalization(xtest1)
xtrain = normalization(xtrain)
xtrain1 = normalization(xtrain1)

woPP=[]; wPP=[]
input_shape1 = (19,1)
input_shape2 = (38,1)
temperature = 0.03
learning_rate=0.001
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
# Splitting xtrain and ytrain into training and validation sets
xtra_a, xval_a, ytra_a, yval_a = train_test_split(xtrain, ytrain, test_size=0.2, random_state=42)

# Splitting xtrain1 and ytrain1 into training and validation sets
xtra_ac, xval_ac, ytra_ac, yval_ac = train_test_split(xtrain1, ytrain1, test_size=0.2, random_state=42)
def create_classifier(hidden_units):

    # for layer in encoder.layers:
    #     layer.trainable = trainable

    inputs = keras.Input(shape=input_shape)
    # features = encoder(inputs)
    features=layers.Dense(hidden_units, activation="relu")(inputs)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units//2, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units//2, activation="relu")(features)
    features = layers.Dense(hidden_units//2, activation="relu")(features)
    features = layers.Dense(hidden_units//2, activation="relu")(features)
    features = layers.Dense(hidden_units//2, activation="relu")(features)
    outputs = layers.Dense(1, activation="sigmoid")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="classifier")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=[keras.metrics.BinaryAccuracy()],
    )
    return model

units=64

classifier = create_classifier(units)

# history = classifier.fit(x=xtra_ac, y=ytra_ac, validation_data =(xval_ac,yval_a), batch_size=batch_size, epochs=num_epochs, callbacks=[EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)])

classifier.summary()
classifier.fit(x=xtra_ac, y=ytra_ac, validation_data =(xval_ac,yval_a), batch_size=batch_size, epochs=num_epochs, callbacks=[EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)])
from sklearn.metrics import accuracy_score

y_pred = classifier.predict(xtest_ac)
y_pred = np.round(y_pred).flatten()
accuracy = accuracy_score(ytest, y_pred)

from sklearn.metrics import f1_score

f1 = f1_score(ytest, y_pred)

print("Accuracy:", accuracy)
print("F1 Score:", f1)

dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])
dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])
dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])
dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])
Model: "classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_71 (InputLayer)       [(None, 38)]              0         
                                                                 
 dense_116 (Dense)           (None, 64)                2496      
                                                                 
 dropout_52 (Dropout)        (None, 64)                0         
                                                                 
 dense_117 (Dense)           (None, 32)                2080      
                          

/usr/local/lib/python3.8/dist-packages/keras/backend.py:5703: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


276/279 [============================>.] - ETA: 0s - loss: 0.5647 - binary_accuracy: 0.7115

/usr/local/lib/python3.8/dist-packages/keras/backend.py:5703: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


279/279 [==============================] - 4s 6ms/step - loss: 0.5645 - binary_accuracy: 0.7115 - val_loss: 0.4816 - val_binary_accuracy: 0.7817
Epoch 2/200
279/279 [==============================] - 2s 5ms/step - loss: 0.4834 - binary_accuracy: 0.7695 - val_loss: 0.4308 - val_binary_accuracy: 0.8068
Epoch 3/200
279/279 [==============================] - 2s 7ms/step - loss: 0.4521 - binary_accuracy: 0.7987 - val_loss: 0.4123 - val_binary_accuracy: 0.8212
Epoch 4/200
279/279 [==============================] - 2s 6ms/step - loss: 0.4277 - binary_accuracy: 0.8059 - val_loss: 0.3932 - val_binary_accuracy: 0.8248
Epoch 5/200
279/279 [==============================] - 2s 6ms/step - loss: 0.4070 - binary_accuracy: 0.8198 - val_loss: 0.3844 - val_binary_accuracy: 0.8302
Epoch 6/200
279/279 [==============================] - 2s 5ms/step - loss: 0.3922 - binary_accuracy: 0.8290 - val_loss: 0.3734 - val_binary_accuracy: 0.8446
Epoch 7/200
279/279 [==============================] - 2s 6ms/step - l

# Combined

In [4]:
fatyp = 'TypicalFA_comb1'
drivepath = 'finalData/'+ fatyp +'/';
#featFiles = 'GER_train_fisher-2000_FA_GT_ESTphnTrans_estStress'               #glob.glob(drivepath + '*train*')
filee = drivepath+'GER_train_fisher-2000_FA_GT_ESTphnTrans_estStress.mat'
data = scipy.io.loadmat(filee)
print(data.keys())
AF = data['AF']
x1 = AF[:-2]
y = AF[-2]
w = AF[-1]
x2 = data['CF']
x = np.concatenate((x1, x2), axis=0)

dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])


In [5]:
fatyp = 'TypicalFA_comb1'
drivepath = 'finalData/'+ fatyp +'/';
#featFiles = 'GER_train_fisher-2000_FA_GT_ESTphnTrans_estStress'               #glob.glob(drivepath + '*train*')
filee = drivepath+'ITA_train_fisher-2000_FA_GT_ESTphnTrans_estStress.mat'
data = scipy.io.loadmat(filee)
print(data.keys())
AF = data['AF']
x3 = AF[:-2]
y1 = AF[-2]
w1 = AF[-1]
x4 = data['CF']
x5 = np.concatenate((x3, x4), axis=0)

dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])


In [11]:
xfinal = np.concatenate((x, x5), axis=1)

In [13]:
xfinal.shape

(38, 11455)

In [16]:
yfinal=np.concatenate((y, y1), axis=None)

In [22]:
print(x(,:5564))

SyntaxError: invalid syntax (3831474976.py, line 1)

In [ ]:
# indices_arr1 = [0, 1, 4, 5, 7]

# # Create a mask to select the specified features from arr1
# mask_arr1 = np.zeros_like(x, dtype=bool)
# mask_arr1[:, indices_arr1] = True

# # Create a mask to select the remaining features from arr2
# mask_arr2 = ~mask_arr1

# # Create the new array by combining selected features from arr1 and arr2
# new_array = np.where(mask_arr1, x, x5)

# # Print the shape of the new array
# print(new_array.shape)

In [26]:
# xfinal=np.concatenate((x, x5), axis=1)


In [27]:
# print(xfinal.shape)

(38, 11455)


In [39]:
# yfinal=np.concatenate((y,y1),axis=0)

In [40]:
# yfinal.shape


(11455,)

In [42]:
np.swapaxes(yfinal,1,0)

AxisError: axis1: axis 1 is out of bounds for array of dimension 1

In [43]:
yfinal

array([0., 1., 0., ..., 1., 0., 0.])

In [44]:
xfinal

array([[0.06249927, 0.05942424, 0.03385573, ..., 0.18293962, 0.15400188,
        0.0673501 ],
       [0.07567894, 0.20975891, 0.03232541, ..., 0.15583806, 0.14611988,
        0.06193826],
       [0.05082466, 0.07201191, 0.02582735, ..., 0.12746012, 0.13397337,
        0.05902464],
       ...,
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ]])

In [35]:

train_path = filee; test_path = filee.replace('train','test')
# print('test file:::::::',os.path.basename(test_path))
xtrain, ytrain, wtrain, info_train = dataloader(train_path, featType); 
xtrain1, ytrain1, wtrain1, info_train1 = dataloader(train_path, featType=2);
xtest, ytest, wtest ,info_test = dataloader(test_path, featType)
xtest1, ytest1, wtest1, info_test1 = dataloader(test_path, featType=2);


xtest_a = normalization(xtest)
xtest_ac = normalization(xtest1)
xtrain = normalization(xtrain)
xtrain1 = normalization(xfinal)

woPP=[]; wPP=[]
input_shape1 = (19,1)
input_shape2 = (38,1)
temperature = 0.03
learning_rate=0.001
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
# Splitting xtrain and ytrain into training and validation sets
xtra_a, xval_a, ytra_a, yval_a = train_test_split(xtrain, ytrain, test_size=0.2, random_state=42)

# Splitting xtrain1 and ytrain1 into training and validation sets
xtra_ac, xval_ac, ytra_ac, yval_ac = train_test_split(xtrain1, yfinal, test_size=0.2, random_state=42)
def create_classifier(hidden_units):

    # for layer in encoder.layers:
    #     layer.trainable = trainable

    inputs = keras.Input(shape=input_shape)
    # features = encoder(inputs)
    features=layers.Dense(hidden_units, activation="relu")(inputs)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units//2, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units//2, activation="relu")(features)
    features = layers.Dense(hidden_units//2, activation="relu")(features)
    features = layers.Dense(hidden_units//2, activation="relu")(features)
    features = layers.Dense(hidden_units//2, activation="relu")(features)
    outputs = layers.Dense(1, activation="sigmoid")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="classifier")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=[keras.metrics.BinaryAccuracy()],
    )
    return model

units=64

classifier = create_classifier(units)

# history = classifier.fit(x=xtra_ac, y=ytra_ac, validation_data =(xval_ac,yval_a), batch_size=batch_size, epochs=num_epochs, callbacks=[EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)])

classifier.summary()
classifier.fit(x=xtra_ac, y=ytra_ac, validation_data =(xval_ac,yval_a), batch_size=batch_size, epochs=num_epochs, callbacks=[EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)])
from sklearn.metrics import accuracy_score

y_pred = classifier.predict(xtest_ac)
y_pred = np.round(y_pred).flatten()
accuracy = accuracy_score(ytest, y_pred)

from sklearn.metrics import f1_score

f1 = f1_score(ytest, y_pred)

print("Accuracy:", accuracy)
print("F1 Score:", f1)

dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])
dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])
dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])
dict_keys(['__header__', '__version__', '__globals__', 'AF', 'AF_info', 'CF', 'CF_info'])


ValueError: Found input variables with inconsistent numbers of samples: [38, 11455]